In [1]:
"""
Import the important libraries for testing the environment
Check whether all the game has been imported
print the dcitionary comprising of all the games

"""

import gym
import random
envs = gym.envs.registry.all()
print(envs)
print('Total envs available:', len(envs))

dict_values([EnvSpec(ALE/Tetris-v5), EnvSpec(ALE/Tetris-ram-v5), EnvSpec(ALE/Adventure-v5), EnvSpec(ALE/Adventure-ram-v5), EnvSpec(ALE/AirRaid-v5), EnvSpec(ALE/AirRaid-ram-v5), EnvSpec(ALE/Alien-v5), EnvSpec(ALE/Alien-ram-v5), EnvSpec(ALE/Amidar-v5), EnvSpec(ALE/Amidar-ram-v5), EnvSpec(ALE/Assault-v5), EnvSpec(ALE/Assault-ram-v5), EnvSpec(ALE/Asterix-v5), EnvSpec(ALE/Asterix-ram-v5), EnvSpec(ALE/Asteroids-v5), EnvSpec(ALE/Asteroids-ram-v5), EnvSpec(ALE/Atlantis-v5), EnvSpec(ALE/Atlantis-ram-v5), EnvSpec(ALE/Atlantis2-v5), EnvSpec(ALE/Atlantis2-ram-v5), EnvSpec(ALE/Backgammon-v5), EnvSpec(ALE/Backgammon-ram-v5), EnvSpec(ALE/BankHeist-v5), EnvSpec(ALE/BankHeist-ram-v5), EnvSpec(ALE/BasicMath-v5), EnvSpec(ALE/BasicMath-ram-v5), EnvSpec(ALE/BattleZone-v5), EnvSpec(ALE/BattleZone-ram-v5), EnvSpec(ALE/BeamRider-v5), EnvSpec(ALE/BeamRider-ram-v5), EnvSpec(ALE/Berzerk-v5), EnvSpec(ALE/Berzerk-ram-v5), EnvSpec(ALE/Blackjack-v5), EnvSpec(ALE/Blackjack-ram-v5), EnvSpec(ALE/Bowling-v5), EnvSpec(AL

In [2]:
"""
Create the open gym environment using gym.make and store the required values such as height, width, actions

"""

env = gym.make('Pong-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

print(height,width,channels)
print(actions)

210 160 3
6


In [3]:
# get the type of actions for the game

env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
#Test the environment to check if the environment is working
# We can see the untrained model gives negative score for each episode


episodes = 5
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
        if done:
            print('Episode: {} Score: {}'.format(episode, score))
            break
            
env.close()

Episode: 1 Score: -20.0
Episode: 2 Score: -21.0
Episode: 3 Score: -19.0
Episode: 4 Score: -21.0
Episode: 5 Score: -20.0


In [5]:
#import the important libraries

import gym
import cv2
from collections import deque,namedtuple
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense
from keras.optimizers import Adam
from numba import jit, cuda

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for d in devices:
    t = d.device_type
    name = d.physical_device_desc
    l = [item.split(':',1) for item in name.split(", ")]
    name_attr = dict([x for x in l if len(x)==2])
    dev = name_attr.get('name', 'Unnamed device')
    print(f" {d.name} || {dev} || {t} || {sizeof_fmt(d.memory_limit)}")

 /device:CPU:0 || Unnamed device || CPU || 256.0 MiB
 /device:GPU:0 ||  NVIDIA GeForce RTX 3050 Ti Laptop GPU || GPU || 1.6 GiB


In [8]:
conv = namedtuple('Conv', 'filter kernel stride')

In [9]:
import tensorflow as tf

In [10]:
GPUS = ["GPU:0"]
strategy = tf.distribute.MirroredStrategy(GPUS)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [11]:
#Initializing replay memory to capacity N

class Memory:
    def __init__(self,size):
        self.size = size
        self.memory = deque()
    def add(self,state,action,reward,next_state,terminal):
        state = np.stack((state[0],state[1],state[2],state[3]),axis=2)
        next_state = np.stack((next_state[0],next_state[1],next_state[2],next_state[3]),axis=2)
        if len(self.memory) < self.size:
            self.memory.appendleft((state,action,reward,next_state,terminal))
        else:
            self.memory.pop()
            self.memory.appendleft((state,action,reward,next_state,terminal))
    def sample(self,batch_size):
        return random.sample(self.memory,batch_size)

In [12]:
#building the agent to train the model and predict the values

class DQN:
    def __init__(self,mem,batch_size=32,mem_size=10000,gamma=0.99,learning_rate=1e-03):
        self.memory = mem
        self.min_mem = mem_size
        self.batch_size = batch_size
        self.gamma = gamma
        self.model = create_network(learning_rate)
        self.target_model = create_network(learning_rate)
        self.copy_network()

    def train_model(self):            
        if len(self.memory.memory) < self.min_mem:
            return
        states,actions,rewards,next_states,terminal = map(np.array,zip(*self.memory.sample(self.batch_size)))
        next_state_action_values = np.max(self.target_model.predict(next_states),axis=1)
        targets = self.model.predict(states)
        targets[range(self.batch_size), actions] = rewards + self.gamma*next_state_action_values*np.invert(terminal)
        self.model.train_on_batch(states, targets)

    def copy_network(self):
        frm = self.model
        to = self.target_model
        for l_tg,l_sr in zip(to.layers,frm.layers):
            wk = l_sr.get_weights()
            l_tg.set_weights(wk)

    def predict(self,state_):
        state1 = np.stack((state_[0],state_[1],state_[2],state_[3]),axis=2)
        return self.model.predict(np.array([state1]))

In [13]:
#create the model with convolution2D and Sequential libraries
# Adam optimizer is used to update the network weights iterative based on training
@jit
def create_network(learning_rate,conv_info=[conv(32,8,4),conv(64,4,2),conv(64,3,1)],dense_info=[512],input_size=(80,80,4)):
    model = Sequential()
    for i,cl in enumerate(conv_info):
        if i==0:
            model.add(Conv2D(cl.filter,cl.kernel,padding="same",strides=cl.stride,activation="relu", input_shape=input_size))
        else:
            model.add(Conv2D(cl.filter,cl.kernel,padding="same",strides=cl.stride,activation="relu"))
    model.add(Flatten())
    for dl in dense_info:
        model.add(Dense(dl,activation="relu"))
    model.add(Dense(6))
    model.compile(loss='mse',optimizer=Adam(learning_rate=learning_rate))
    return model

In [14]:
#creating the environment to play various episoed and produce sample actions

class Pong:
    def __init__(self):
        self.env = gym.make('Pong-v0')
        self.eps = 1
        self.memory = Memory(50000)
        self.dqn = DQN(self.memory)
        self.copy_period = 40000
        self.itr = 0
        self.eps_step = 0.0000009

    def sample_action(self,state):
        if random.random() < self.eps:
            return self.env.action_space.sample()
        return np.argmax(self.dqn.predict(state)[0])


    def play_one_episode(self):
        observation = self.env.reset()
        done = False
        state = []
        update_state(state,observation)
        prv_state = []
        total_reward = 0
        while not done:

            if len(state) < 4:
                action = self.env.action_space.sample()
            else:
                action = self.sample_action(state)
        
            prv_state.append(state[-1])
            if len(prv_state) > 4:
                prv_state.pop(0)
            observation, reward, done, _ = self.env.step(action)

            update_state(state,observation)
            if len(state) == 4 and len(prv_state) == 4:
                self.memory.add(prv_state,action,reward,state,done)
            total_reward += reward

            self.itr += 1
            if self.itr % 4 == 0:
                self.dqn.train_model()
            self.eps = max(0.1,self.eps-self.eps_step)
            if self.itr % self.copy_period == 0:
                self.dqn.copy_network()
        return total_reward

In [15]:
#resizing and converting image from one color space to another

def downsample(observation):
    s = cv2.cvtColor(observation[30:,:,:], cv2.COLOR_BGR2GRAY)
    s = cv2.resize(s, (80,80), interpolation = cv2.INTER_AREA) 
    s = s/255.0
    return s

In [16]:
#update the states

def update_state(state,observation):
    ds_observation = downsample(observation)
    state.append(ds_observation)
    if len(state) > 4:
        state.pop(0)

In [17]:
p = Pong()
for i in range(5000):
    total_reward = p.play_one_episode()
    print("episode total reward:",total_reward)
    if i%100 == 0:
        print("Saving the model")
        p.dqn.model.save("model-{}.h5".format(i))

C:\Users\agarw\AppData\Local\Temp/ipykernel_21136/2900727641.py:3: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "create_network" failed type inference due to: Untyped global name 'Sequential': Cannot determine Numba type of <class 'type'>

File "AppData\Local\Temp\ipykernel_21136\2900727641.py", line 5:
<source missing, REPL/exec in use?>

  @jit
C:\Users\agarw\AppData\Local\Temp/ipykernel_21136/2900727641.py:3: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "create_network" failed type inference due to: Untyped global name 'Sequential': Cannot determine Numba type of <class 'type'>

File "AppData\Local\Temp\ipykernel_21136\2900727641.py", line 5:
<source missing, REPL/exec in use?>

  @jit
C:\Users\agarw\anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "create_network" was compiled in object mode without forceobj=True, but has lifted lo

episode total reward: -20.0
Saving the model
1/1 [==============================] - 0s 16ms/step
episode total reward: -19.0
1/1 [==============================] - 0s 23ms/step
episode total reward: -20.0
1/1 [==============================] - 0s 29ms/step
episode total reward: -21.0
1/1 [==============================] - 0s 24ms/step
episode total reward: -20.0
1/1 [==============================] - 0s 16ms/step
episode total reward: -19.0
1/1 [==============================] - 0s 32ms/step
episode total reward: -20.0
1/1 [==============================] - 0s 24ms/step
episode total reward: -20.0
1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 24ms/step


1/1 [==============================] - 0s 25ms/step


...

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 43ms/step


1/1 [==============================] - 0s 66ms/step


1/1 [==============================] - 0s 40ms/step


KeyboardInterrupt: 

In [18]:
def sample_action(model,state):
    return np.argmax(model.predict(np.array([np.stack((state[0],state[1],state[2],state[3]),axis=2)]))[0])

In [23]:
from keras.models import load_model
env = gym.make('ALE/Pong-v5')
model = load_model('model.h5')


done = False
state = []
observation = env.reset()
score = 0
update_state(state,observation)

while not done:
#     env.render()
    if len(state) < 4:
        action = env.action_space.sample()
    else:
        action = sample_action(model,state)
    observation, reward, done, _ = env.step(action)
    score+=reward
    update_state(state,observation)

1/1 [==============================] - 0s 49ms/step


1/1 [==============================] - 0s 89ms/step


1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 53ms/step


1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 42ms/step


1/1 [==============================] - 0s 48ms/step


1/1 [==============================] - 0s 62ms/step


**Establish a baseline performance.
How well did your Deep Q-learning do on your problem? (5 Points)Links to an external site. For example
total_episodes = 5000
total_test_episodes = 100
max_steps = 99
learning_rate = 0.7
gamma = 0.8
epsilon = 1.0
max_epsilon = 1.0
min_epsilon = 0.01
decay_rate = 0.01 With this baseline performance, our RL program with the Taxi-v2 Toy text gives us a score of 8.13, which is considered not bad.**

Before the model was trained, we were getting negative scores for the player, but after training, we got the positive score
What are the states, the actions, and the size of the Q-table? (5 Points)
Actions - 6 - ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']
States - [ball_direction, ball_position, paddle_one, paddle_two]
The size of Q table is 24
  
**What are the rewards? Why did you choose them? (5 Points)**
	So, to win the game a player should score 21 points. It's simply a +1 for every game the agents win and -1 for the loses for the opponent.
The reason to chose them is to get the high positive score to get the winner by the end of the game as each steps cannot be rewarded.
 
**How did you choose alpha and gamma in the Bellman equation? Try at least one additional value for alpha and gamma. How did it change the baseline performance? (5 Points)**
Alpha is the learning rate that has been set to 1e-03 and chosen randomly as it performed quicker learning of the model as compared to 2.5e-4
Gamma is set to 0.99 
Setting the values to the above mentioned value significantly improved the performance

 
**Try a policy other than e-greedy. How did it change the baseline performance? (5 Points)**
Some papers mentioned the use of Softmax policy apart from e-greedy for deep q learning algorithms whihc can be used via LinearAnealed
policy library.
 
How did you choose your decay rate and starting epsilon? Try at least one additional value for epsilon and the decay rate. How did it change the baseline performance? What is the value of epsilon when you reach the max steps per episode? (5 Points)

 
**What is the average number of steps taken per episode? (5 Points)**

The average number of steps taken per episode was 757 approx.. The average time per step was 35.8ms/step.
Does Q-learning use value-based or policy-based iteration? (5 Points)
Q learning uses value-based iteration, which uses the Q value to
Explain, not a yes or no question. 

**Could you use SARSA for this problem? (5 Points)**

    For the fast iterative action, Deep Q learning is preferred as it focuses on the total reward following egreedy appraoch. SARSA can also be used for this problem, but it is expected to give poor performance as compared to Q learning.
 
**What is meant by the expected lifetime value in the Bellman equation?**

    The expected life time value in Bellman equation tells the agent how good the action is at a given state by following certain policy. This value is drived by gamma value which mostly is 0.9 as 
    (5 Points)
    Explain, not a yes or no question. 
 
**When would SARSA likely do better than Q-learning? (5 Points)
Explain, not a yes or no question.**

    If the user doesnt want their unexpected failure to be costly, then SARSA can be used for this problem as it an on policy algorithm which learns from the action performed at current state to get the Q vlaue.

**How does SARSA differ from Q-learning? (5 Points)  
Details include pseudocode and math.**

    SARSA is on policy algorithm whereas Q leanrning is model free, value based off policy algorithm.
    Q learning focuses on the total reward and then perform the action based on that reward, whereas the SARSA learns from the actions performed in the current state and learns the Q value following e greedy algothim.

 
**Explain the Q-learning algorithm. (5 Points)**
        Q-Learning is a Reinforcement learning policy that will find the next best action, given a current state. It chooses this action at random and aims to maximize the reward.
        In the Q-Learning algorithm, the goal is to learn iteratively the optimal Q-value function using the Bellman Optimality Equation. So, we store the Q value and update it repetitively at each step using the Q learning iteration 
        V(x)=\max _{a\in \Gamma (x)}\{F(x,a)+\beta V(T(x,a))\}.

    Initialize replay memory D to capacity N
    Initialize action-value function Q with random weights
    for episode = 1, M do
    Initialise sequence s1 = {x1} and preprocessed sequenced φ1 = φ(s1)
    for t = 1, T do
    With probability  select a random action at
    otherwise select at = maxa Q∗
    (φ(st), a; θ)
    Execute action at in emulator and observe reward rt and image xt+1
    Set st+1 = st, at, xt+1 and preprocess φt+1 = φ(st+1)
    Store transition (φt, at, rt, φt+1) in D
    Sample random minibatch of transitions (φj , aj , rj , φj+1) from D
    Set yj =
    
    rj for terminal φj+1
    rj + γ maxa0 Q(φj+1, a0
    ; θ) for non-terminal φj+1
    Perform a gradient descent step on (yj − Q(φj , aj ; θ))2
    according to equation 3
    end for
    end for



 
**Explain the SARSA algorithm. (5 Points)**  
Details include pseudocode and math. 
    SARSA is the on policy algorithm which uses e greedy algorithm to learn from the action performed by the current state to learn Q value.
    SARSA is an on-policy algorithm where, in the current state, S an action, A is taken and the agent gets a reward, R and ends up in next state, S1 and takes action, A1 in S1. Therefore, the tuple (S, A, R, S1, A1) stands for the acronym SARSA.
    Q(s,a)=Q(s,a)+α[r+γQ(s′,a′)−Q(s,a))]

    Pseudo code for sarsa:
    Initialize Q(s,a) arbitrarily
    Repeat (for each episode):
       Initialize s
       Choose a from s using a policy derived from Q (e.g., ɛ-greedy)
       Repeat (for each step of the episode):
          Take action a, observe r, s'
          Choose a' from s' using policy derived from Q (e.g., ɛ-greedy)
          Q (s,a) ← Q(s,a) + α[r + γQ(s',a') - Q(s,a)]
          s ← s'; a ← a'
       until s is terminal

    alpha represents the learning rate, gamma represents the discounted rate and r represents the rewards, the algorithm gets after performing action a from state s leading to state s’.
    Q(s,a) stores the value of doing action a from state s. There will be 36 states and 4 different actions (1 = going up, 2 = left, 3 = down, 4 = right).
 
**What code is yours, and what have you adapted? (5 Points)**

    You must explain what code you wrote and what you have done that is different. Failure to cite ANY code will result in a zero for this section.

    The optimization part and testing environment are mine, and another part of the code is adapted from the several sources mentioned below. The code was segregated and made modular for better understanding. I tried optimizing the code using Cuda by automatically running it on GPU using TensorFlow GPU


    References:
    https://github.com/jeffheaton/present/blob/master/youtube/gpu/keras-dual-gpu.ipynb
    https://towardsdatascience.com/intro-to-reinforcement-learning-pong-92a94aa0f84d
    https://github.com/amirhossein-hkh/pong-dqn
    https://github.com/philtabor/Deep-Q-Learning-Paper-To-Code

16. Did I explain my code clearly? (10 Points)
Yes
 
17. Did I explain my licensing? (5 Points)
Yes.

 
18. Professionalism (10 Points)
Variable naming, style guide, conduct, behavior, and attitude.
I have tried following the coding standards by defining proper variable names and comments for each class or function.
 
